In [ ]:
!pip install prettytable==3.9.0 --quiet
import os
os.kill(os.getpid(), 9)  # Restart the runtime after installing

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Director.csv to Director.csv
Saving Film_Director.csv to Film_Director.csv
Saving Film_Genre.csv to Film_Genre.csv
Saving Films.csv to Films.csv
Saving Genre.csv to Genre.csv
Saving Users.csv to Users.csv
Saving Viewer_Stats.csv to Viewer_Stats.csv


In [ ]:
import pandas as pd

films = pd.read_csv("films.csv")
directors = pd.read_csv("director.csv")
film_directors = pd.read_csv("film_director.csv")
genres = pd.read_csv("genre.csv")
film_genres = pd.read_csv("film_genre.csv")
users = pd.read_csv("users.csv")
viewer_stats = pd.read_csv("viewer_stats.csv")

In [ ]:
import sqlite3
conn = sqlite3.connect("movies.db")  # Creates a persistent DB file
cursor = conn.cursor()

In [ ]:
cursor = conn.cursor()

cursor.executescript("""
DROP TABLE IF EXISTS films;
CREATE TABLE films (
  film_id INTEGER PRIMARY KEY,
  film_title TEXT,
  release_year INTEGER,
  duration INTEGER,
  budget DECIMAL(12,2),
  revenue DECIMAL(12,2),
  ROI_percent DECIMAL(6,2),
  decade TEXT,
  language TEXT
);

DROP TABLE IF EXISTS directors;
CREATE TABLE directors (
  director_id INTEGER PRIMARY KEY,
  name TEXT
);

DROP TABLE IF EXISTS film_directors;
CREATE TABLE film_directors (
  film_id INTEGER,
  director_id INTEGER,
  PRIMARY KEY (film_id, director_id),
  FOREIGN KEY (film_id) REFERENCES films(film_id),
  FOREIGN KEY (director_id) REFERENCES directors(director_id)
);

DROP TABLE IF EXISTS genres;
CREATE TABLE genres (
  genre_id INTEGER PRIMARY KEY,
  genre_name TEXT
);

DROP TABLE IF EXISTS film_genres;
CREATE TABLE film_genres (
  film_id INTEGER,
  genre_id INTEGER,
  PRIMARY KEY (film_id, genre_id),
  FOREIGN KEY (film_id) REFERENCES films(film_id),
  FOREIGN KEY (genre_id) REFERENCES genres(genre_id)
);

DROP TABLE IF EXISTS users;
CREATE TABLE users (
  user_id INTEGER PRIMARY KEY,
  age INTEGER,
  age_group TEXT,
  state TEXT,
  subscription_type TEXT
);

DROP TABLE IF EXISTS viewer_stats;
CREATE TABLE viewer_stats (
  user_id INTEGER,
  film_id INTEGER,
  watch_date DATE,
  rating DECIMAL(5,2),
  watch_duration_minutes INTEGER,
  FOREIGN KEY (user_id) REFERENCES users(user_id),
  FOREIGN KEY (film_id) REFERENCES films(film_id)
);
""")
conn.commit()

In [ ]:
print(users.columns)
films.rename(columns={"ROI": "ROI_percent"}, inplace=True)
genres.drop(columns=["Unnamed: 2", "Unnamed: 3", "Unnamed: 4", "Unnamed: 5", "Unnamed: 6"], inplace=True)

viewer_stats['watch_date'] = pd.to_datetime(viewer_stats['watch_date'], errors='coerce')


Index(['user_id', 'age', 'age_group', 'state', 'subscription_type'], dtype='object')


In [ ]:
films.to_sql("films", conn, index=False, if_exists="append")
directors.to_sql("directors", conn, index=False, if_exists="append")
film_directors.to_sql("film_directors", conn, index=False, if_exists="append")
genres.to_sql("genres", conn, index=False, if_exists="append")
film_genres.to_sql("film_genres", conn, index=False, if_exists="append")
users.to_sql("users", conn, index=False, if_exists="append")
viewer_stats.to_sql("viewer_stats", conn, index=False, if_exists="append")

5000

In [ ]:
%load_ext sql
%sql sqlite:///movies.db

In [ ]:
%%sql
SELECT film_title, ROI_percent, decade FROM films LIMIT 5;

 * sqlite:///movies.db
Done.


film_title,ROI_percent,decade
Spirited Away,"1,347",2000s
My Neighbor Totoro,"1,008",1980s
Princess Mononoke,619,1990s
Howl's Moving Castle,884,2000s
Kiki's Delivery Service,-35,1980s


In [ ]:
#1: "Which films perform best by viewer rating and watch completion?"

%%sql
SELECT
    f.film_title,
    COUNT(vs.user_id) AS total_views,
    ROUND(AVG(vs.rating), 2) AS avg_rating,
    ROUND(AVG(vs.watch_duration_minutes * 1.0 / f.duration) * 100, 1) AS avg_completion_percent
FROM
    viewer_stats vs
JOIN
    films f ON vs.film_id = f.film_id
GROUP BY
    f.film_title
ORDER BY
    total_views DESC;

 * sqlite:///movies.db
Done.


film_title,total_views,avg_rating,avg_completion_percent
Tales from Earthsea,232,46.0,62.7
Ponyo,220,83.0,85.7
My Neighbors the Yamadas,213,75.0,73.7
When Marnie Was There,211,88.0,81.6
The Secret World of Arrietty,210,85.0,78.2
Earwig and the Witch,210,68.0,65.7
Spirited Away,209,96.0,66.7
Howl's Moving Castle,208,93.0,56.0
The Boy and the Heron,206,88.0,61.9
The Cat Returns,205,82.0,62.3


In [ ]:
#2: "Which genres are most popular by age group?"

%%sql
SELECT
    g.genre_name,
    CASE
        WHEN u.age < 18 THEN 'Teen'
        WHEN u.age BETWEEN 18 AND 34 THEN 'Young Adult'
        WHEN u.age BETWEEN 35 AND 54 THEN 'Adult'
        ELSE 'Senior'
    END AS age_group,
    COUNT(*) AS total_views,
    ROUND(AVG(vs.rating), 2) AS avg_rating
FROM
    viewer_stats vs
JOIN users u ON vs.user_id = u.user_id
JOIN film_genres fg ON vs.film_id = fg.film_id
JOIN genres g ON fg.genre_id = g.genre_id
GROUP BY genre_name, age_group
ORDER BY total_views DESC;

 * sqlite:///movies.db
Done.


genre_name,age_group,total_views,avg_rating
Drama,Senior,1050,82.96
Fantasy,Adult,769,81.02
Adventure,Senior,702,84.14
Drama,Young Adult,498,84.75
Family,Adult,454,86.43
Fantasy,Young Adult,363,80.4
Adventure,Young Adult,334,83.1
Family,Young Adult,238,87.43
Drama,Teen,205,93.25
Adventure,Teen,140,79.84


In [ ]:
#3: "Which directors have the highest average viewer rating?"

%%sql
SELECT
    d.name AS director_name,
    ROUND(AVG(vs.rating), 2) AS avg_viewer_rating,
    COUNT(DISTINCT vs.film_id) AS num_films_rated,
    COUNT(vs.user_id) AS total_ratings
FROM
    viewer_stats vs
JOIN film_directors fd ON vs.film_id = fd.film_id
JOIN directors d ON fd.director_id = d.director_id
GROUP BY
    d.name
HAVING
    COUNT(vs.user_id) > 30 -- filters out directors with low sample size
ORDER BY
    avg_viewer_rating DESC;

 * sqlite:///movies.db
Done.


director_name,avg_viewer_rating,num_films_rated,total_ratings
Hayao Miyazaki,90.03,11,2166
Hiromasa Yonebayashi,86.5,2,421
Isao Takahata,84.21,6,1198
Hiroyuki Morita,82.0,1,205
Yoshifumi Kondo,76.83,2,379
Goro Miyazaki,63.51,3,631


In [ ]:
#4: "How does viewer behavior differ by subscription type?"

%%sql
SELECT
    u.subscription_type,
    COUNT(vs.user_id) AS total_views,
    ROUND(AVG(vs.rating), 2) AS avg_rating,
    ROUND(AVG(vs.watch_duration_minutes * 1.0 / f.duration) * 100, 1) AS avg_completion_percent
FROM
    viewer_stats vs
JOIN users u ON vs.user_id = u.user_id
JOIN films f ON vs.film_id = f.film_id
GROUP BY
    u.subscription_type
ORDER BY
    total_views DESC;

 * sqlite:///movies.db
Done.


subscription_type,total_views,avg_rating,avg_completion_percent
Family,1438,84.13,69.4
Premium,1351,82.47,71.2
Free,1142,84.32,70.0
Student,1069,83.83,70.6


In [ ]:
#5: "What is the ROI (Return on Investment) per film?"

%%sql
SELECT
    film_title,
    budget,
    revenue,
    ROI_percent
FROM
    films
ORDER BY
    roi_percent DESC;

 * sqlite:///movies.db
Done.


film_title,budget,revenue,ROI_percent
Spirited Away,"19,000,000.00","274,925,095.00","1,347"
My Neighbor Totoro,"3,700,000.00","41,000,000.00","1,008"
Howl's Moving Castle,"24,000,000.00","236,049,757.00",884
My Neighbors the Yamadas,"20,500,000.00","167,000,000.00",715
Princess Mononoke,"23,500,000.00","169,000,000.00",619
The Secret World of Arrietty,"23,000,000.00","149,480,483.00",550
Whisper of the Heart,"7,000,000.00","34,900,000.00",399
Ponyo,"34,000,000.00","153,000,000.00",350
The Wind Rises,"30,000,000.00","117,932,401.00",293
Porco Rosso,"9,200,000.00","34,100,000.00",271


In [ ]:
#6: "Are there seasonal trends in viewership?"... (#EXTRACT(MONTH FROM watch_date) AS month)

%%sql
SELECT
    strftime('%m', watch_date) AS month,
    COUNT(*) AS total_views,
    ROUND(AVG(rating), 2) AS avg_rating,
    ROUND(AVG(watch_duration_minutes), 1) AS avg_watch_time
FROM
    viewer_stats
GROUP BY
    month
ORDER BY
    month;

 * sqlite:///movies.db
Done.


month,total_views,avg_rating,avg_watch_time
01,423,84.8,77.4
02,362,81.81,75.3
03,401,81.75,75.5
04,413,82.2,73.9
05,446,85.17,75.5
06,398,85.21,73.8
07,415,80.83,73.8
08,421,82.1,76.9
09,417,83.89,74.2
10,436,84.65,79.0


In [ ]:
#7: "Which U.S. states have the highest average viewer rating for Ghibli films — but only among highly active users?"

%%sql
SELECT
    u.state,
    ROUND(AVG(vs.rating), 2) AS avg_rating,
    COUNT(vs.film_id) AS total_views
FROM
    viewer_stats vs
JOIN users u ON vs.user_id = u.user_id
WHERE
    vs.user_id IN (
        SELECT user_id
        FROM viewer_stats
        GROUP BY user_id
        HAVING COUNT(film_id) >= 10
    )
GROUP BY
    u.state
ORDER BY
    avg_rating DESC
LIMIT 10;

 * sqlite:///movies.db
Done.


state,avg_rating,total_views
Washington,102.45,10
California,100.45,10
Colorado,97.03,11
Vermont,95.66,25
Nebraska,95.23,11
Wyoming,94.32,16
Massachusetts,90.88,11
Pennsylvania,90.26,21
Virginia,89.69,10
Iowa,83.58,21


In [ ]:
#8: "Who are the top 3 most active users in each U.S. state by number of films watched?"

%%sql
WITH user_activity AS (
    SELECT
        u.user_id,
        u.state,
        COUNT(vs.film_id) AS total_films_watched
    FROM
        users u
    JOIN viewer_stats vs ON u.user_id = vs.user_id
    WHERE u.state IS NOT NULL
    GROUP BY u.user_id, u.state
),
ranked_users AS (
    SELECT
        user_id,
        state,
        total_films_watched,
        ROW_NUMBER() OVER (
            PARTITION BY state
            ORDER BY total_films_watched DESC
        ) AS state_rank
    FROM user_activity
    WHERE total_films_watched >= 5
)
SELECT *
FROM ranked_users
WHERE state_rank <= 3
ORDER BY state, state_rank;

 * sqlite:///movies.db
Done.


user_id,state,total_films_watched,state_rank
300,Alabama,11,1
107,Alabama,9,2
474,Alabama,9,3
133,Alaska,9,1
481,Alaska,8,2
798,Alaska,8,3
151,Arizona,6,1
245,Arizona,6,2
936,Arizona,6,3
421,Arkansas,10,1
